In [20]:
import math
import pandas as pd
import networkx as nx
from itertools import combinations

edges = pd.read_csv("./駅データ/join20240426.csv")
stations = pd.read_csv("./駅データ/station20240426free.csv")
line = pd.read_csv("./駅データ/line20240426free.csv", usecols=["line_cd", "line_name"])

In [21]:
stations[stations["line_cd"] == 1001]

,station_cd,station_g_cd,station_name,station_name_k,station_name_r,line_cd,pref_cd,post,address,lon,lat,open_ymd,close_ymd,e_status,e_sort


In [2]:
stations = stations.merge(line, how='left')

In [3]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [4]:
def get_station_cd(station_name):
    station_cd = stations.loc[stations["station_name"] == station_name, "station_cd"]
    if not station_cd.empty:
        return station_cd.iloc[0]
    else:
        raise ValueError(f"No station found with name {station_name}")

def simplify_path(path):
    if not path:
        return []
    simplified_path = [path[0]]
    current_line = G.nodes[path[0]]['line_cd']
    for station in path[1:]:
        next_line = G.nodes[station]['line_cd']
        if next_line != current_line:
            simplified_path.append(station)
            current_line = next_line
    if path[-1] != simplified_path[-1]:
        simplified_path.append(path[-1])
    return simplified_path


In [5]:
edges = edges.merge(stations.loc[:,["station_cd", "lon", "lat"]], how='left', left_on='station_cd1', right_on='station_cd', suffixes=('', '_1'))
edges.drop(columns=['station_cd'], inplace=True)
edges.rename(columns={'lon': 'lon1', 'lat': 'lat1'}, inplace=True)

edges = edges.merge(stations.loc[:,["station_cd", "lon", "lat"]], how='left', left_on='station_cd2', right_on='station_cd', suffixes=('', '_2'))
edges.drop(columns=['station_cd'], inplace=True)
edges.rename(columns={'lon': 'lon2', 'lat': 'lat2'}, inplace=True)
edges = edges.dropna()

edges['distance'] = edges.apply(lambda row: haversine(row["lat1"], row["lon1"], row["lat2"], row["lon2"]), axis=1)
edges['weight'] = edges['distance'].apply(lambda x: math.exp(x / 2))



In [7]:
G = nx.Graph()

for idx, row in stations.iterrows():
    G.add_node(row['station_cd'], name=row['station_name'] + " " + row["line_name"], lat=row['lat'], lon=row['lon'], line_cd=row['line_cd'])

for idx, row in edges.iterrows():
    src_cd = row['station_cd1']
    dst_cd = row['station_cd2']
    weight = row["weight"]
    G.add_edge(src_cd, dst_cd, weight=weight)

grouped_stations = stations.groupby('station_g_cd')

for group_cd, group in grouped_stations:
    station_pairs = combinations(group['station_cd'], 2)
    for src_cd, dst_cd in station_pairs:
        G.add_edge(src_cd, dst_cd, weight=edges['weight'].max() * 2)


In [8]:
def debug_path(path_cd):
    print("経路デバッグ情報:")
    for station_cd in path_cd:
        station = G.nodes[station_cd]
        print(f"駅コード: {station_cd}, 駅名: {station['name']}, 路線コード: {station['line_cd']}, 緯度: {station['lat']}, 経度: {station['lon']}")


In [9]:
source_name = "横浜"
target_name = "大阪"

try:
    source = get_station_cd(source_name)
    target = get_station_cd(target_name)
    
    shortest_path_cd = nx.shortest_path(G, source=source, target=target, weight='weight')
    simplified_path_cd = simplify_path(shortest_path_cd)
    
    shortest_path_names = [G.nodes[data]['name'] for data in shortest_path_cd]
    simplified_path_names = [G.nodes[data]['name'] for data in simplified_path_cd]
    
    # print("最短経路（駅名）:", " -> ".join(shortest_path_names))
    print("簡略化された経路（駅名）:\n", " -> \n".join(simplified_path_names))

except nx.NetworkXNoPath:
    print("経路が見つかりません。")


簡略化された経路（駅名）:
 横浜 JR東海道本線(東京～熱海) -> 
熱海 JR東海道本線(熱海～浜松) -> 
浜松 JR東海道本線(浜松～岐阜) -> 
岐阜 JR東海道本線(岐阜～美濃赤坂・米原) -> 
米原 琵琶湖線 -> 
京都 JR京都線 -> 
大阪 JR京都線


In [22]:
shinkansen_lines = line[line['line_name'].str.contains('新幹線')]


In [19]:
stations[stations["line_cd"] == 1001]

,station_cd,station_g_cd,station_name,station_name_k,station_name_r,line_cd,pref_cd,post,address,lon,lat,open_ymd,close_ymd,e_status,e_sort,line_name


In [23]:
shinkansen_lines

,line_cd,line_name
0,1001,中央新幹線
1,1002,東海道新幹線
2,1003,山陽新幹線
3,1004,東北新幹線
4,1005,上越新幹線
5,1006,上越新幹線(ガーラ湯沢支線)
6,1007,山形新幹線
7,1008,秋田新幹線
8,1009,北陸新幹線
9,1010,九州新幹線


In [10]:
raise

RuntimeError: No active exception to reraise

In [ ]:
i = 0
for (u, v, data) in G.edges(data=True):
    print(f"Edge from {u} to {v} has weight: {data['weight']}")
    if i >= 10:
        break
    i+=1

In [ ]:
path = nx.shortest_path(G, source=1110101 , target=1110107, weight='weight')
print("Shortest path from {} to {} is: {}".format(source, target, path))


In [ ]:
get_station_cd("新横浜")

In [ ]:
stations[stations["station_name"] == "新横浜"]